## Case Study 1
#### Note: This should be completed individually.

Assume you are new to the data science field, and you want to find out what real practitioners and soon-to-be data scientists are concerned about. One place where you may find such information is X (formerly known as Twitter). However, X users often use their real identities and may have reservations about sharing all their opinions publicly. Another place where such information maybe found is the datascience subreddit on Reddit.com (https://www.reddit.com/r/datascience/). Users are assumed to be anonymous and they are more likely to share their opinions without reservations. To find out common concerns among the datascience subreddit users, it might be a good idea to collect the top 100 posts in the subreddit in the year 2024. You might also collect the top 3 comments of each of those posts. In this case study, we will do exactly that. Specific details can be found in the next few cells. 

The collected dataset can be used for many different projects. However, we are only going to focus on the "data gathering" part. We will also do some cleaning.

**Note**: This case study contributes 10% to your overall grade.

## Step 1: 
###  15 points


**Description:** 

Learn about the **PRAW** package for Python and learn how you can use it to load reddit posts, comments etc. on a Jupyter Notebook. Do a Google search. You may find tutorials. It is okay to use them. You may need to use secret keys for this part. For that you will need to open a Reddit account. Please use a throwaway account for this purpose (the TA and the Prof will use your credentials when grading). 

Write your code in the cell below. Any code you write to retrieve data from Reddit can go there.

Here is a link to the PRAW documentation: https://praw.readthedocs.io/en/stable/#getting-started

**Grading criteria:** 

The code for this step must be correct. Otherwise, the next steps cannot be completed. In that case, the next steps will not be graded. If you receive a praw object from the data science subreddit, you will get full 15 points.' Other methods may be considered, but not encouraged.

In [42]:
import praw
import pandas as pd
from datetime import datetime

# Reddit API credentials
client_id = "sweVEQRduZ02ugrUX8M37g"
client_secret = "WoTD17PDVqc2SgrNKGdmtw6DhoQkgQ"
user_agent = "windows:script for case study 1r:v1.0 (by u/GrowthSuitable4972)"

# Create Reddit instance
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=user_agent)

# Get the subreddit
subreddit = reddit.subreddit('datascience')

# Collect top posts
top_posts = []
for post in subreddit.top(limit=500, time_filter='year'):
    post_time = datetime.fromtimestamp(post.created_utc)
    if post_time.year == 2024 and post.score > 50:
        # Only consider posts with more than 5 words in the title
        if len(post.title.split()) > 5:
            top_posts.append({
                'Date': post_time.strftime('%Y-%m-%d'),
                'Post score': post.score,
                'Post title': post.title,
                'Body of the post': post.selftext,
                'Post ID': post.id
            })

    # Stop when we have at least 100 posts
    if len(top_posts) >= 100:
        break

# Convert posts to a DataFrame
df_posts = pd.DataFrame(top_posts)
df_posts.head()


,Date,Post score,Post title,Body of the post,Post ID
0,2024-01-31,1708,Friendly reminder not to work too hard. You'll...,The year just started and there are already ov...,1afgvv9
1,2024-02-27,1388,Data scientist quits her job at Spotify,In summary and basically talks about how she w...,1b1au2f
2,2024-02-19,1062,The BS they tell about Data Science…,1. In what world does a Director of DS only ma...,1augdle
3,2024-03-05,890,Everything I've been doing is suddenly conside...,Anyone else experience this where your company...,1b7ano8
4,2024-03-10,796,Turns out my best data science work is helping...,,1bbqqus


## Step 2: 
### 10 + 20 + 10 + 15 + 5 + 5 = 65 points

**Description**:
Once you have the mechanism in place to retrieve data from Reddit, you next step is to determine which parts of the data is necessary. For this case study, collect only the top posts from the year 2024. Also consider if the score of each post was above 50 or not. If the score was below 50, it might not have been an important post. Do not consider those posts. 

You may also observe that sometimes posts with memes or jokes get a lot of 'upvotes,' and because of that they may  have high scores, but they may not be useful for this case study. To address this problem, you will simply get rid of any post that has fewer than 5 words in the title. 

You will also notice that praw returns time as an integer. It is inconvenient for us to read time like that. You may want to convert the integer time to human readable time. You do not need to mention hours, minutes, or seconds. Just year, month and date is enough.

**Grading Criteria:**
* posts are only from the year 2024: 10 points
* the integer time format converted into year-month-day: 20 points
* only posts with scores more than 50 were considered: 10 points
* only post titles with more than 5 words were kept: 15 points
* minimum 100 posts were collected: 5 points
* post title, body of the post (if present), and three comments collected for each post: 5 points

Note: All six grading criteria can be satified by writing one line or many lines of code. It does not matter. As long as your code satisfies the six criteria (in one line or many lines), you will get full points.

Also note: In case the API does not allow you to collect 100 posts, you can collect 80 or 90 etc. In that case, please copy and paste the error message in a new cell below.

In [46]:

posts_with_comments = []

for post_id in df_posts['Post ID']:
    submission = reddit.submission(id=post_id)
    submission.comments.replace_more(limit=0)
    top_comments = submission.comments[:3]  # Get top 3 comments

    # Store the top 3 comments or empty strings if fewer than 3
    comments = [comment.body if idx < len(top_comments) else '' for idx, comment in enumerate(top_comments)]
    while len(comments) < 3:  # Add empty comments if there are fewer than 3
        comments.append('')

    # Add the post info along with the top 3 comments
    post_info = {
        'Date': submission.created_utc,
        'Post score': submission.score,
        'Post title': submission.title,
        'Body of the post': submission.selftext,
        'Top comment 1': comments[0],
        'Top comment 2': comments[1],
        'Top comment 3': comments[2]
    }
    posts_with_comments.append(post_info)

    df_posts_with_comments = pd.DataFrame(posts_with_comments)
    df_posts_with_comments['Date'] = pd.to_datetime(df_posts_with_comments['Date'], unit='s').dt.strftime('%Y-%m-%d')
df_posts_with_comments.head()

,Date,Post score,Post title,Body of the post,Top comment 1,Top comment 2,Top comment 3
0,2024-01-31,1707,Friendly reminder not to work too hard. You'll...,The year just started and there are already ov...,"I'm in HR Workforce management,\n\nLet me tell...",I agree. I'm going through something similar a...,"I lost my job three times in a row, so I think..."
1,2024-02-27,1387,Data scientist quits her job at Spotify,In summary and basically talks about how she w...,You can’t manage a pushy stakeholder unless yo...,> How do you manage a “pushy” stakeholder?\n\n...,"This is not uncommon, especially for data scie..."
2,2024-02-19,1056,The BS they tell about Data Science…,1. In what world does a Director of DS only ma...,Wait... who's paying Analysts 90k🧐,The real bs is we can start right here lol,Meanwhile to anyone outside of the US these sa...
3,2024-03-05,888,Everything I've been doing is suddenly conside...,Anyone else experience this where your company...,Ride the wave 🌊 just make sure to have savings...,"IMO, the greatest achievement of AI is compani...","10+ years ago it was all ""Big Data"".\n\nMost o..."
4,2024-03-11,798,Turns out my best data science work is helping...,,I’ve mentored a few friends into getting bette...,Are you looking to help more people lol?,What would be your advice for a new grad seeki...


## Step 3: 
### 10 points

Save the data on your local disk. You may have used lists or similar data structures for the intial porcessing. Convert that data structure into a Pandas dataframe. Save the dataframe as a .csv file into your local disk. 

Here are the column details:

Column 1: Date

Column 2: Post score

Column 3: Post title

Column 4: Body of the post (if present)

Column 5: Top comment 1

Column 6: Top comment 2

Column 7: Top comment 3

When you create the .csv file, it should have at least 101 rows (including column names) and 7 columns.

**Grading criteria:**
If your code produces a .csv file in the local disk in the same folder as the Jupyter Notebook file, you get full points. Otherwise, no points.

In [48]:
# Convert to DataFrame
df_posts_with_comments = pd.DataFrame(posts_with_comments)

# Save the DataFrame to a CSV file
df_posts_with_comments.to_csv(r'C:\Users\sylve\Downloads\sylvester_krampah_casestudy_1\sylvester_krampah_casestudy_1.csv', index=False)

# Print confirmation
print("CSV file saved successfully with top 3 comments.")

CSV file saved successfully with top 3 comments.


## Step 4:
### 10 points
#### Presentation slides:
   
Create presentation slides for this case study. The presentation slides should provide an overview of the problem you tried to solve, methods you have used (don't put actual code in the slides), and if you have discovered new insights from the data you have collected. You may put actual post titles or comments in the slide that you found insightful and interesting. The number of slides should be around 6-7 (no hard limit). Three of you will be randomly chosen and be asked to present your work in the class. You should be prepared to present your work for 5 mins.


**Notes on grading**: 5 points will be deducted if you are not prepared to present on the day of submission.

### What to submit:

All files should be named in the following format:

firstname_lastname_casestudy_1.pdf

firstname_lastname_casestudy_1.ipynb

firstname_lastname_casestudy_1.csv

etc.


Put the Jupyter Notebook file and the .csv file in a folder. Then convert your presentation slides to a PDF file and put it in the same folder. Zip the folder. After zipping, it should have the extension .zip. The name of the .zip file should be firstname_lastname_casestudy_1.zip . Upload the .zip file on Canvas.

